In [3]:
# コマンド定義


import sys
import time
import math
import copy
sys.path.append("pybcapclient")
import pybcapclient.bcapclient as bcapclient

class Cobotta():
    def __init__(self) -> None:

        self.COMP = 1
        
        host = "192.168.56.11"
        port = 5007
        timeout = 2000

        ### Connection processing of tcp communication
        self.m_bcapclient = bcapclient.BCAPClient(host,port,timeout)
        
    def connect(self):
        ### start b_cap Service
        self.m_bcapclient.service_start("")
        ### Connect to RC8 (RC8(VRC)provider)
        Name = ""
        Provider="CaoProv.DENSO.VRC"
        Machine = ("localhost")
        Option = ("")
        hCtrl = self.m_bcapclient.controller_connect(Name,Provider,Machine,Option)
        self.hCtrl=hCtrl
        ### get Robot Object Handl
        HRobot = self.m_bcapclient.controller_getrobot(hCtrl,"Arm","")
        self.HRobot=HRobot

        self.m_bcapclient.robot_execute(self.HRobot, "ManualResetPreparation", "")
        self.m_bcapclient.robot_execute(self.HRobot, "Motionpreparation", "")

        ### TakeArm
        Command = "TakeArm"
        Param = [0,0]
        self.m_bcapclient.robot_execute(self.HRobot,Command,Param)

        ###Motor On
        Command = "Motor"
        Param = [1,0]
        self.m_bcapclient.robot_execute(self.HRobot,Command,Param)

        ###set ExtSpeed Speed,Accel,Decel
        Command = "ExtSpeed"
        Speed = 100
        Accel = 100
        Decel = 100
        Param = [Speed,Accel,Decel]
        self.m_bcapclient.robot_execute(self.HRobot,Command,Param)

    def disconnect(self):
        Command = "Motor"
        Param = [0,0]
        self.m_bcapclient.robot_execute(self.HRobot,Command,Param)

        ###Give Arm
        Command = "GiveArm"
        Param = None
        self.m_bcapclient.robot_execute(self.HRobot,Command,Param)
        #Disconnect
        if(self.HRobot != 0):
            self.m_bcapclient.robot_release(self.HRobot)
        #End If
        if(self.hCtrl != 0):
            self.m_bcapclient.controller_disconnect(self.hCtrl)
        #End If
        self.m_bcapclient.service_stop()
        
        print('Disconnection succeeded')

    def auto_calibration(self):
        try:
            self.disconnect()
        except:
            pass
        # Connection processing of tcp communication
        self.m_bcapclient = bcapclient.BCAPClient(host, port, timeout)
        print("Open Connection")

        # start b_cap Service
        self.m_bcapclient.service_start("")
        print("Send SERVICE_START packet")

        # set Parameter
        Name = ""
        Provider = "CaoProv.DENSO.VRC"
        Machine = "localhost"
        Option = ""

        try:
            # Connect to RC8 (RC8(VRC)provider) , Get Controller Handle
            self.hCtrl = self.m_bcapclient.controller_connect(Name, Provider, Machine, Option)
            print("Connect RC8")
            # Get Robot Handle
            self.HRobot = self.m_bcapclient.controller_getrobot(self.hCtrl, "Arm", "")

            self.m_bcapclient.robot_execute(self.HRobot, "AutoCal", "")
            self.m_bcapclient.robot_execute(self.HRobot, "MotionPreparation", "")
            self.m_bcapclient.robot_execute(self.HRobot, "TakeArm")
            self.m_bcapclient.robot_execute(self.HRobot, "ExtSpeed", 100)
            self.m_bcapclient.robot_move(self.HRobot, 1, "@P J(0,0,90,0,90,0)")
            time.sleep(1)
           


        except Exception as e:
            print('=== ERROR Description ===')
            if str(type(e)) == "<class 'pybcapclient.orinexception.ORiNException'>":
                print(e)
                errorcode_int = int(str(e))
                if errorcode_int < 0:
                    errorcode_hex = format(errorcode_int & 0xffffffff, 'x')
                else:
                    errorcode_hex = hex(errorcode_int)
                print("Error Code : 0x" + str(errorcode_hex))
                error_description = self.m_bcapclient.controller_execute(
                    self.hCtrl, "GetErrorDescription", errorcode_int)
                print("Error Description : " + error_description)
            else:
                print(e)

        finally:
            # DisConnect
            if(self.HRobot != 0):
                self.m_bcapclient.robot_release(self.HRobot)
                print("Release Robot Handle")
            # End If
            if(self.hCtrl != 0):
                self.m_bcapclient.controller_disconnect(self.hCtrl)
                print("Release Controller")
            # End If
            self.m_bcapclient.service_stop()
            print("B-CAP service Stop")
            print("Auto Calibration Completed")

    def change_speed(self,speed=100,accel=100,decel=100):
        Command = "ExtSpeed"
        Param = [speed,accel,decel]
        self.m_bcapclient.robot_execute(self.HRobot,Command,Param)
        print("Speed Parameters ","speed:",speed,"accel:",accel,"decel:",decel)

    def get_now_position(self):
        return self.m_bcapclient.robot_execute(self.HRobot, "CurPos")

    def get_now_joint(self):
        return self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    def clear_error(self):
        self.m_bcapclient.controller_execute(self.hCtrl,"ManualResetPreparation","")
        self.m_bcapclient.controller_execute(self.hCtrl,"ClearError","")

    def motion_preparation(self):
        self.m_bcapclient.robot_execute(self.HRobot, "ManualResetPreparation", "")
        self.m_bcapclient.robot_execute(self.HRobot, "Motionpreparation", "")

    def get_error_information(self):
        return self.m_bcapclient.controller_execute(self.hCtrl,"GetCurErrorInfo",0)[0]

    def get_eroor_count(self):
        return self.m_bcapclient.controller_execute(self.hCtrl,"GetCurErrorCount",0)

    def get_cobotta_status(self):
        pos=self.get_now_position()
        if 280<pos[0]<300 and -50<pos[1]<-40 and 10<pos[2]<20 and -100<pos[3]<-80 and -10<pos[4]<10 and -100<pos[5]<-80:
            return "Sleep"
        else:
            return "Active"

    # IO
    def read_IO_value(self,IO_No):
        IOHandl = 0
        IOHandl = self.m_bcapclient.controller_getvariable(self.hCtrl, f"IO{IO_No}", "")
        # read value
        retIO = self.m_bcapclient.variable_getvalue(IOHandl)
        # Disconnect
        if(IOHandl != 0):
            self.m_bcapclient.variable_release(IOHandl)
        self.m_bcapclient.service_stop()
        return retIO

    def write_IO_value(self,IO_No,IO_Value):
        IOHandl = 0
        IOHandl = self.m_bcapclient.controller_getvariable(self.hCtrl, f"IO{IO_No}", "")

        # write value
        self.m_bcapclient.variable_putvalue(IOHandl, IO_Value)
        # read value
        retIO = self.m_bcapclient.variable_getvalue(IOHandl)
        # Disconnect
        if(IOHandl != 0):
            self.m_bcapclient.variable_release(IOHandl)
        self.m_bcapclient.service_stop()
        return retIO


    # hand
    def open_hand(self):
        param=[29.91,50]
        self.m_bcapclient.controller_execute(self.hCtrl,"HandMoveA",param)
        print("Hand Open")

    def close_hand(self,force=6):
        param=[force,True,"DetectOn"]
        self.m_bcapclient.controller_execute(self.hCtrl,"HandMoveH",param)
        print("Hand Close")

    def adjust_hand(self):
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")
        if now_pos[5]>-45:
            self.move_joint6(-90)
        else:
            self.move_joint6(90)


    # move
    def go_to_position_parameter(self,param):
        Pose = [param,"P","@E"]
        self.m_bcapclient.robot_move(self.HRobot,self.COMP,Pose,"")

    def go_to_position_parameter_at_P(self,param):
        Pose = [param,"P","@P"]
        self.m_bcapclient.robot_move(self.HRobot,self.COMP,Pose,"")

    def go_to_joint_parameter(self,param):
        Pose = [param,"J","@E"]
        self.m_bcapclient.robot_move(self.HRobot,self.COMP,Pose,"")

    def go_to_position_variable(self,P_No):
        Pose = "P"+str(P_No)
        self.m_bcapclient.robot_move(self.HRobot,self.COMP,Pose,"")
        return self.m_bcapclient.robot_execute(self.HRobot, "CurPos")

    def go_to_joint_variable(self,J_No):
        Joint = "J"+str(J_No)
        self.m_bcapclient.robot_move(self.HRobot,self.COMP,Joint,"")
        return self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    def shift_parallel(self,x=0,y=0,z=0,rx=0,ry=0,rz=0):
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurPos")
        target_pos=[now_pos[0]+x,now_pos[1]+y,now_pos[2]+z,now_pos[3]+rx,now_pos[4]+ry,now_pos[5]+rz]
        Pose = [target_pos,"P","@E"]
        self.m_bcapclient.robot_move(self.HRobot,self.COMP,Pose,"")
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurPos")

    def move_joint1(self,param):
        joint_pos=[param]
        param=[joint_pos,"J","@E"]
        self.m_bcapclient.robot_execute(self.HRobot, "DriveEx",param)
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    def move_joint2(self,param):
        joint_pos=[0,param,0,0,0,0]
        param=[joint_pos,"J","@E"]
        self.m_bcapclient.robot_execute(self.HRobot, "DriveEx",param)
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    def move_joint3(self,param):
        joint_pos=[0,0,param,0,0,0]
        param=[joint_pos,"J","@E"]
        self.m_bcapclient.robot_execute(self.HRobot, "DriveEx",param)
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    def move_joint4(self,param):
        joint_pos=[0,0,0,param,0,0,0]
        param=[joint_pos,"J","@E"]
        self.m_bcapclient.robot_execute(self.HRobot, "DriveEx",param)
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    def move_joint5(self,param):
        joint_pos=[0,0,0,0,param,0]
        param=[joint_pos,"J","@E"]
        self.m_bcapclient.robot_execute(self.HRobot, "DriveEx",param)
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")


    def move_joint6(self,param):
        joint_pos=[0,0,0,0,0,param]
        param=[joint_pos,"J","@E"]
        self.m_bcapclient.robot_execute(self.HRobot, "DriveEx",param)
        now_pos= self.m_bcapclient.robot_execute(self.HRobot, "CurJnt")

    
    def move_in_a_spiral(self,center,radius_max,rotation_number):
        for i in range(0,360*rotation_number,10):
            x = center[0] + radius_max/(rotation_number*2*math.pi)*math.radians(i)*math.cos(math.radians(i))
            y = center[1] + radius_max/(rotation_number*2*math.pi)*math.radians(i)*math.sin(math.radians(i))
            self.go_to_position_parameter_at_P([x,y,center[2]-10,center[3],center[4],center[5],center[6]])
        self.shift_parallel(z=5)


   

In [36]:
cobotta=Cobotta()
cobotta.connect()

# move from calset pose
cobotta.move_joint1(-20)
cobotta.move_joint2(20)
cobotta.move_joint3(-20)
cobotta.move_joint4(20)
cobotta.move_joint5(-20)
cobotta.move_joint6(-180)

cobotta.disconnect()

ORiNException: -2091904988

In [34]:
cobotta=Cobotta()
cobotta.connect()
cobotta.move_joint5(-10)
cobotta.disconnect()

ORiNException: -2091904988

In [37]:

cobotta.clear_error()
cobotta.disconnect()

Disconnection succeeded
